In [3]:
#%%time

import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

got=0
pitching=True
lst=[]
consecutive=0
n=0

skip=20

#IMPORT TEMPLATES AND GRAB DIMENSIONS
box = cv2.imread('E:/HOU18/game 2/snapshots/full box.jpg',0)  
height, width = box.shape

p=cv2.imread('E:/HOU18/game 2/snapshots/pitch p.jpg',0) 
height_p, width_p = p.shape

#IMPORT VIDEO
cap = cv2.VideoCapture('E://HOU18/game 2/HOU AT TEX - March 30, 2018 video.mp4')


if cap.isOpened()== False: 
    print("Error opening the video file. Please double check your file path for typos")

while cap.isOpened():
    
    cap.set(cv2.CAP_PROP_POS_FRAMES, n)
    # Read a new frame
    ret, frame = cap.read()
    
    if ret == True:
        
        grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        #FIND PITCH COUNT P
        res_p =cv2.matchTemplate(grayFrame,p,cv2.TM_CCOEFF_NORMED)
        min_val_p, max_val_p, min_loc_p, max_loc_p = cv2.minMaxLoc(res_p)
        
        top_left_p = max_loc_p
        bottom_right_p = (top_left_p[0] + width_p, top_left_p[1] + height_p) 

        #see if p: is in correct location
        if top_left_p[1] == 499:
            consecutive+=1
            cv2.rectangle(frame,top_left_p, bottom_right_p,(0,255,0), 1)
            
            # if consecutive frames have the p in correct location take time
            if consecutive==7:

                cv2.putText(frame, "pitch recorded", (1014,478), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,255,0),2)
                got+=1
                t=cap.get(cv2.CAP_PROP_POS_MSEC)
                t=int(np.floor(t/1000))
                lst.append([got,t])
            
            # pitching continues past recorded moment
            elif consecutive > 7:
                cv2.putText(frame, "pitch recorded", (1014,478), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,255,0),2)

        # p is not being detected in correct location
        else:
            consecutive=0
            
         #Display the resulting frame
        cv2.imshow('frame', frame)
        
        # Exit if ESC pressed
        k = cv2.waitKey(1) & 0xff
        if k == 27 : 
            break
    else:
        break
        
    n= n+skip
    
# When everything is done, release the capture
cap.release()
cv2.destroyAllWindows()

# write list to csv file
df=pd.DataFrame(lst,columns =['Pitch number','seconds'])
df.to_csv('test.csv',index=False)